In [325]:
import pretty_midi as pm
import matplotlib.pyplot as plt
# Env variables
chrom_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
offsets = {
    '1': 0,
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11,
    '8': 12,
    '9': 14,
    '10': 16,
    '11': 17,
    '12': 19,
    '13': 21
}

In [326]:
# Returns the first note by time in a list of notes
def first_note(notes):
    if notes == []:
        return None
    f_note = notes[0]
    for i in range(1, len(notes)):
        try:
            if notes[i].start < f_note.start:
                f_note = notes[i].start
        except (AttributeError):
            print("Bad")
    return f_note

# Returns the last note by time in a list of notes
def last_note(notes):
    if notes == []:
        return None
    l_note = notes[0]
    for i in range(1, len(notes)):
        try:
            if notes[i].end > l_note.end:
                l_note = notes[i].end
        except (AttributeError):
            print("Bad")
    return l_note

# Returns a list of all the non-drum notes in a song regardless of instrument
def consolidate_notes(song):
    notes = []
    for instrument in song.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.add(note)
    return notes

# Returns a note name based on its MIDI note number
def get_note(note_n):
    return chrom_notes[note_n % 12]

# Returns the correct note based on a root note of a scale and its degree
def parse_chord(root, number_string):
    note_num = chrom_notes.index(root)
    out = ""
    num = ""
    scale_num = 0
    parentheses = False
    for char in number_string:
        if char == '(':
            parentheses = True
        if char == 'b':
            scale_num -= 1
        if char == '#':
            scale_num += 1
        if char >= '0' and char <= '9':
            num += char
    scale_num += offsets.get(num)
    if (parentheses):
        out = "("
    out += str(get_note(note_num + scale_num))
    if (parentheses):
        out += ")"
    return out

# Outputs the master chord list (dict version)
def generate_chord_list(filepath = "chords without names.txt"):
    chord_list = []
    for note in chrom_notes:
        f = open(filepath)
        lines = f.readlines()
        for line in lines:
            parts = line.split()
            chord_name = ''
            note_list = []
            for i in range(len(parts)):
                part = parts[i]
                if i == 0:
                    chord_name = part.replace('_', note, 1)
                elif part[0] == 'b' or part[0] == '#' or \
                   (part[0] >= '0' and part[0] <= '9') or \
                   part[0] == '(':
                    note_list.append(parse_chord(note, part))
                else: continue
            chord_list.append([chord_name, note_list])
    return chord_list

# Gets the chords in a song
def get_chords(notes, 
               offset = 0.01):
    start_times = []
    for note in notes:
        if not (note.start in start_times):
            start_times.append(note.start)
    chords = []
    for time in start_times:
        playing_notes = []
        actual = time + offset
        for note in notes:
            if note.start < actual and note.end >= time:
                song.instruments[0].notes
                playing_notes.append(note)
        chords.append(playing_notes)
    return chords

# Generates note scores for the piece
def get_note_scores(notes, 
                    octave_multiplier_on = False,
                    end_multiplier_on = False,
                    printVals = False):
    note_scores_octave_agn = []
    note_scores_octave_agn_dict = dict()
    last_end = last_note(notes).end
    first_start = first_note(notes).start
    overall_dur = last_end - first_start
    for i in range(0, 12):
        note_scores_octave_agn.append(0)
    for note in notes:
        duration = note.end - note.start
        score = duration * note.velocity / 127
        octave_multiplier = 1
        if octave_multiplier_on:
            octave_multiplier = max(0, 1 - (max(0, (round(note.pitch % 12) - 3) / 8.0)))
            print("%d %f" % (note.pitch, octave_multiplier))
        if end_multiplier_on:
            end_multiplier = note.end / last_end
        else:
            end_multiplier = 1
        score *= octave_multiplier
        score *= end_multiplier
        note_scores_octave_agn[note.pitch % 12] += score
    for i in range(0, 12):
        if note_scores_octave_agn[i] != 0:
            note_scores_octave_agn_dict[i] = note_scores_octave_agn[i]
    if printVals:
        note_scores_octave_agn_sorted = sorted(note_scores_octave_agn_dict.items(), key=lambda x:x[1], reverse = True)
        print(inst.name)
        for key in note_scores_octave_agn_dict.keys():
            print("Note %s has a score of %f" % (get_note(key), note_scores_octave_agn_dict[key]))
        print("The most common note is %s" % get_note(note_scores_octave_agn_sorted[0][0]))
        print("------")
        print()
    else:
        return note_scores_octave_agn_dict, overall_dur, last_end, first_start
    
# Generates chord scores based on note scores
def get_chord_scores(chord_list, 
                     note_scores_octave_agn_dict, 
                     overall_dur,
                     parentheses_multiplier = 1,
                     min_note_threshold = 0.1, 
                     missing_deweight = 0.5, 
                     printVals = False):
    chord_scores_dict = {}
    for chord_tuple in chord_list:
        chord_name = chord_tuple[0]
        chord_notes = chord_tuple[1]
        chord_score = 0.0
        for note in chord_notes:
            multiplier = 1
            actual_note = note
            if note[0] == '(':
            #    multiplier = parentheses_multiplier
                actual_note = note[1 : (len(note) - 1)]
            note_val = chrom_notes.index(actual_note)
            note_score = note_scores_octave_agn_dict.get(note_val, 0)
            if note_score <= min_note_threshold:
                note_score = -1 * missing_deweight # Deweight chords with missing notes
            chord_score += note_score * multiplier
        if chord_score > 0.0:
            chord_scores_dict[chord_name] = chord_score
    chord_scores_dict_sorted = sorted(chord_scores_dict.items(), key=lambda x:x[1], reverse = True)
    if printVals:
        print("The 10 highest-scoring chords are:")
        for i in range(10):
            print("%d: %s with a score of %f" % ((i + 1), chord_scores_dict_sorted[i][0], chord_scores_dict_sorted[i][1]))
    else:
        return chord_scores_dict_sorted

# Makes a list of all the chords in a song
def calculate_song_chords(song):
    all_chords = generate_chord_list()
    chord_list = []
    for chord in get_chords(song.instruments[0].notes):
        note_dict, overall_dur, last_end, first_start = get_note_scores(chord)
        chord_list.append(get_chord_scores(all_chords, note_dict, last_end)[:1][0][0])
    return chord_list

# Returns n-grams of the items of a list (used in this case to make chord-grams)
def n_grams(my_list, n):
    items = []
    for i in range(0, len(my_list) - n):
        n_gram = []
        for j in range(i, i + n):
            n_gram.append(my_list[j])
        items.append(n_gram)
    return items

# Returns the number of chord changes per second on average of a song
def chord_changes(chord_list, song):
    notes = song.instruments[0].notes
    duration = last_note - first_start
    chord_changes_per_time = (len(chord_list) - 1) / duration
    return chord_changes_per_time

In [327]:
song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\gen hoshino-Piano.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.midi")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\progression.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Untitled score.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\7-Strings Ensemble Staccato 7 test.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\jazz but without the piano chords.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\blind comp render E-PIANO ONLY_basic_pitch.mid")

In [328]:
all_chords = generate_chord_list()
for chord in get_chords(song.instruments[0].notes):
    note_dict, overall_dur, last_end, first_start = get_note_scores(chord)
    print(get_chord_scores(all_chords, note_dict, last_end)[:1], end = " @ ")
    print(f"%.2f s" % first_start)

[('Esus2', 2.5435559517060367)] @ 0.00 s
[('Bm', 0.9528388692913388)] @ 1.26 s
[('Do7', 1.2704518257217845)] @ 1.89 s
[('AM9', 3.179444939632546)] @ 2.53 s
[('Co7', 1.2704518257217845)] @ 3.79 s
[('Gb0', 1.0741816062992118)] @ 4.42 s
[('Dbm9', 2.6224618845144345)] @ 4.96 s
[('Cm9', 0.7923747034120727)] @ 6.00 s
[('D6', 1.2704518257217845)] @ 6.32 s
[('E7', 1.2704518257217845)] @ 6.95 s
[('AM9', 0.7923747034120727)] @ 7.58 s
[('Abm7', 0.6338997627296582)] @ 7.89 s
[('Gbm9', 4.770825097112862)] @ 8.21 s
[('AM9', 6.362205254593178)] @ 10.11 s
[('Dsus2', 1.6948198677165331)] @ 12.63 s
[('Dbm9', 4.2403650446194225)] @ 13.47 s
[('Dbsus4', 5.089764203674543)] @ 15.16 s
[('Esus4', 3.180108014698158)] @ 17.68 s
[('Gbsus4', 1.9070038887139091)] @ 19.26 s
[('Esus4', 3.8166600776902935)] @ 20.21 s
[('Abm7', 1.270451825721788)] @ 22.11 s
[('AM9', 4.770825097112858)] @ 22.74 s
[('Gb0', 1.270451825721788)] @ 24.63 s
[('Dbm9', 5.566515175853018)] @ 25.26 s
[('Bm9', 0.7923747034120661)] @ 27.47 s
[('AM

In [329]:
chords = calculate_song_chords(song)
print(chord_changes(chords, song))

38.8321135523117
